In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import holidays as pyholidays

In [3]:
shift_type={"morning":["M", "COM", "DISP"],"afternoon":["T1", "T2", "CHX"]}

# demands dictionary has a first key representing the shift type, a second key representing the day of the week, and a third  separating
# holidays and not holidays using 1 to indicate holiday and 0 to indicate not holiday

def random_demands_generator(shift_type,year:int=2022,month:int=5,max_demand:int=5):

    demands={}
    for shift in shift_type.keys():
        morning_max_demand=np.random.randint(max_demand-1,max_demand+1)
        afternoon_max_demand=np.random.randint(1,morning_max_demand)
        for st in shift_type[shift]:
            for day in range(7):
                for holiday in [0,1]:
                    if shift=="morning":
                        if st=="M":
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=morning_max_demand
                            else:
                                demands[(st,day,holiday)]=morning_max_demand-1
                        elif st=="COM":
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=morning_max_demand-1
                            else:
                                demands[(st,day,holiday)]=morning_max_demand-2
                        else:
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=0
                            else:
                                demands[(st,day,holiday)]=max(morning_max_demand-2,1)
                    if shift=="afternoon":
                        if st=="T1":
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=afternoon_max_demand
                            else:
                                demands[(st,day,holiday)]=afternoon_max_demand
                        elif st=="T2":
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=max(afternoon_max_demand-1,1)
                            else:
                                demands[(st,day,holiday)]=max(afternoon_max_demand-1,0)
                        else:
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=max(morning_max_demand-1,1)  
                            else:
                                demands[(st,day,holiday)]=0                  
    list_shifts=[]
    # list of dates of january 2023
    
    days_in_month = None
    if month == 2:
        days_in_month = 28
    elif month in [4, 6, 9, 11]:
        days_in_month = 30
    else:
        days_in_month = 31
        

    dates = pd.date_range(start=f'{month}/1/{year}', end=f'{month}/{days_in_month}/{year}')
    shift_id=0
    for fecha in dates:
        for shift in shift_type:
            for jornada in shift_type[shift]:
                list_shifts.append([shift_id, 
                                    fecha.strftime("%Y-%m-%d"), 
                                    fecha.week,
                                    jornada, shift,fecha.weekday(),
                                    fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023),
                                    demands[(jornada,fecha.weekday(),int( fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023)))]
                                    ])
                shift_id+=1
    df_shifts=pd.DataFrame(list_shifts,columns=["shift_id","shift_date","week_of_year","shift","shift_type","weekday","holiday","demand"])
    df_shifts.index=df_shifts["shift_id"]
    
    return df_shifts



In [10]:
import numpy as np
import calendar
import datetime
def create_random_instance(n_nurses:int,year:int=2022,month:int=5,seed:int=None):
    """
    Create a random instance of the nurse scheduling problem.

    return: pandas dataframe with columns 'nurse_id', 'nurse_name',  'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours','vacations'
    """
    if seed != None:
        np.random.seed(seed)
    
    nurse_id = np.arange(n_nurses)
    nurse_name = np.array([f'Nurse {i}' for i in nurse_id])
    shift_preference = ['afternoon' if np.random.rand() < 0.5 else 'morning' for i in nurse_id]
    morning_availability_labor_day = np.random.randint(0,2,n_nurses)
    morning_availability_weekend = np.random.randint(0,2,n_nurses)
    afternoon_availability_labor_day = np.random.randint(0,2,n_nurses)
    afternoon_availability_weekend = np.random.randint(0,2,n_nurses)


    #enumerate weekend days of the month
    weekend_days = []
    for i in range(1,calendar.monthrange(year,month)[1]+1):
        if datetime.date(year,month,i).weekday() > 4:
            weekend_days.append(i)
    
    # select random weekend days
    dates_off = []
    for i in range(n_nurses):
        dates_off.append(np.random.choice(weekend_days, size=np.random.randint(0,len(weekend_days)), replace=False))
    accumulated_hours = list(np.random.randint(-30,30,n_nurses))
    vacations = []
    for i in range(n_nurses):
        if np.random.rand() < 0.2:
            vacations_duration = np.random.randint(7,15)
            start_date = np.random.randint(1,calendar.monthrange(year,month)[1]+1)
            vacations.append([start_date+j for j in range(vacations_duration) if start_date+j<32])
        else:
            vacations.append([])
    return pd.DataFrame({'nurse_id':nurse_id,
                            'nurse_name':nurse_name,
                            'shift_preference':shift_preference,
                            'morning_availability_labor_day':morning_availability_labor_day,
                            'morning_availability_weekend':morning_availability_weekend,
                            'afternoon_availability_labor_day':afternoon_availability_labor_day,
                            'afternoon_availability_weekend':afternoon_availability_weekend,
                            'dates_off':dates_off,
                            'accumulated_hours':accumulated_hours,
                            'vacations':vacations})



In [14]:
import os
from os.path import exists
from app.libs.HHCRS_multiobjective_model import check_feasibility
from itertools import product
import math

#for tot_nurses,month,year in product(range(8,12),range(1,13),[2022,2023]):
    #for iter in range(10):
tot_nurses=17
month=5
year=2022

df_shifts=random_demands_generator(shift_type,year,month,math.floor(tot_nurses/2.66))
df_random=create_random_instance(tot_nurses,year,month)
df_random.to_json('tmp/nurses.json',orient='records')
df_shifts.to_json('/Users/user/Documents/HHCRSP/proyecto_HHCRSP/app/libs/shifts.json',orient='records')

"""if check_feasibility('tmp/') :

    # checks whether the folder name already exists, otherwise creates it
    if not exists(f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}'):
        os.mkdir(f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}')

    # checks the number of files in the folder and creates a new one with the next number
    files = os.listdir(f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}')
    if len(files) == 0:
        nurse_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/n_nurses_{tot_nurses}_{month}-{year}_1.json'
        shifts_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/shifts_{month}-{year}_1.json'
    else:
        nurse_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/n_nurses_{tot_nurses}_{month}-{year}_{len(files)+1}.json'
        shifts_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/shifts_{month}-{year}_{len(files)+1}.json'

    df_random.to_json(nurse_file,orient='records')
    df_shifts.to_json(shifts_file,orient='records')"""
            #  move to a new folder named as the details of the instance
    
    
print()